In [19]:
import pandas as pd

class Escola:
    def __init__(self, URE, USE, MUNICIPIO, COD_ESCOLA, COD_DISCIPLINA, CH_HORARIA, VAGAS):
        self.URE = URE
        self.USE = USE
        self.MUNICIPIO = MUNICIPIO
        self.COD_ESCOLA = COD_ESCOLA
        self.COD_DISCIPLINA = COD_DISCIPLINA
        self.CH_HORARIA = CH_HORARIA
        self.VAGAS = VAGAS

# Carregar o DataFrame
base_escolas = pd.read_excel('escolas.xlsx')

# Dicionário para armazenar os objetos Escola
escolas_dict = {}

# Percorrer as linhas do DataFrame
for indice, linha in base_escolas.iterrows():
    # Gerar o nome do objeto
    nome_objeto = f"escola_{linha['COD_ESCOLA']}"
    
    # Criar um objeto Escola para cada linha
    escola = Escola(
        linha['URE'],
        linha['USE'],
        linha['MUNICIPIO'],
        linha['COD_ESCOLA'],
        linha['COD_DISCIPLINA'],
        linha['CH_HORARIA'],
        linha['VAGAS']
    )
    
    # Adicionar o objeto ao dicionário usando o nome como chave
    escolas_dict[nome_objeto] = escola

for nome_objeto, escola in escolas_dict.items():
    print(f"Nome do objeto: {nome_objeto}, VAGAS: {escola.VAGAS}, Disciplina {escola.COD_DISCIPLINA}")



Nome do objeto: escola_1, VAGAS: 2, Disciplina 1
Nome do objeto: escola_2, VAGAS: 2, Disciplina 2003
Nome do objeto: escola_3, VAGAS: 2, Disciplina 2002
Nome do objeto: escola_4, VAGAS: 1, Disciplina 2003
Nome do objeto: escola_10, VAGAS: 3, Disciplina 2002
